In [1]:
import nltk
import csv

In [2]:
data=[]
with open('Words.csv', 'r',encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        data+=[row]

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Yashasvi
[nltk_data]     Rai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yashasvi
[nltk_data]     Rai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def process(word_data):
    New_word=[]
    stop_words=stopwords.words('english')
    New_word = [word for word in word_data if word.lower() not in stop_words]
    for i in range(len(New_word)):
        string=""
        for j in str(New_word[i]):
            if j not in "0123456789":
                string+=j.lower()
            else:
                string+=j
        New_word[i]=string
    pattern = r'\w+'
    tokenizer = RegexpTokenizer(pattern)
    New_word=tokenizer.tokenize(' '.join(New_word))
    stemmer=PorterStemmer()
    word_data=[]
    for i in range(len(New_word)):
        word_data+=[stemmer.stem(New_word[i])]
    return word_data
processed_data=[]
for i in data:
    processed_data+=[process(i)]

In [5]:
Query_tokens=[]
with open('Queries.txt','r') as File:
    Queries=File.readlines()
for i in Queries:
    Query_tokens+=[i.split()]

In [6]:
for i in range(len(Query_tokens)):
    Query_tokens[i]=process(Query_tokens[i])
print(Query_tokens)

[['identifi', 'industri', 'canoo', 'oper', 'along', 'size', 'growth', 'rate', 'trend', 'key', 'player'], ['analyz', 'canoo', 's', 'main', 'competitor', 'includ', 'market', 'share', 'product', 'servic', 'offer', 'price', 'strategi', 'market', 'effort'], ['identifi', 'key', 'trend', 'ev', 'market', 'includ', 'chang', 'consum', 'behavior', 'technolog', 'advanc', 'shift', 'competit', 'landscap'], ['gather', 'inform', 'canoo', 's', 'financi', 'perform', 'includ', 'revenu', 'profit', 'margin', 'return', 'invest', 'expens', 'structur']]


In [7]:
from gensim.models import Word2Vec
import numpy as np
def vector(document, word2vec_model):
    array=[]
    for i in document:
        if i in word2vec_model.wv:
            array+=[word2vec_model.wv[i]]
    return np.mean(array,axis=0)
VectorData=[]
Query_Vectors={}
word2vec_model = Word2Vec(sentences=processed_data, vector_size=60, window=20, min_count=1)
word2vec_model.save("word2vec_model.bin")
for j in processed_data:
    VectorData+=[vector(j,word2vec_model)]
# with open('VectorData.csv','w',newline='') as csvfile:
#     writer=csv.writer(csvfile)
#     writer.writerows(VectorData)
for i in range(len(Query_tokens)):
    Query_Vectors[i]=[vector(Query_tokens[i],word2vec_model)]

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

Dict = {}
for query_id in Query_Vectors.keys():
    query_vector = Query_Vectors[query_id]
    similarities = []
    query_documents = set()
    for j, vector in enumerate(VectorData):
        similarity = cosine_similarity(query_vector, [vector])
        similarities.append((similarity, j))
    similarities.sort(reverse=True)
    Documents = []
    for similarity, j in similarities:
        if data[j] not in Documents:
            Documents.append(data[j])
            query_documents.add(j)
            if len(Documents) == 5:
                break
    Dict[Queries[query_id]] = Documents

In [9]:
for i in Dict:
    temp=[]
    for j in Dict[i]:
        temp+=[' '.join(j)]
    Dict[i]=temp

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [13]:
textfile=open("Solution.txt",'w',encoding='utf-8')
for i in Dict:
    query=i
    Docs=Dict[i]
    Input2=''
    for doc in Docs:
        pattern = r'\w+'
        tokeniz = RegexpTokenizer(pattern)
        doc=tokeniz.tokenize(doc)
        newdoc=' '.join(doc)
        Input2+=newdoc+' '
    Input=f"{query} {Input2}"
    inputs = tokenizer(Input, return_tensors="pt", max_length=1024, truncation=True)
    output = model.generate(inputs.input_ids,attention_mask=inputs.attention_mask, max_length=200, num_return_sequences=1,
                            temperature=0.7, top_k=50, top_p=0.95)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text=generated_text[len(query):]
    textfile.writelines('Query: '+query+'\n')
    textfile.writelines('\n')
    textfile.writelines('Answer:'+generated_text+'\n')
    textfile.writelines('\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
